# Predicting Movie Rental Durations

![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

## Load data

In [2]:
# Load rental info into workspace
rental_info = pd.read_csv("datasets/rental_info.csv")

# View layout of rental_info
rental_info.head()

rental_date                return_date  amount  release_year  \
0  2005-05-25 02:54:33+00:00  2005-05-28 23:40:33+00:00    2.99        2005.0   
1  2005-06-15 23:19:16+00:00  2005-06-18 19:24:16+00:00    2.99        2005.0   
2  2005-07-10 04:27:45+00:00  2005-07-17 10:11:45+00:00    2.99        2005.0   
3  2005-07-31 12:06:41+00:00  2005-08-02 14:30:41+00:00    2.99        2005.0   
4  2005-08-19 12:30:04+00:00  2005-08-23 13:35:04+00:00    2.99        2005.0   

   rental_rate  length  replacement_cost                special_features  \
0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
1         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
2         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
3         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
4         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   

   NC-17  PG  PG-13  R  amount_2  length_2  rental_rate_2  
0      0   0      0  1    8.9401   15876.0         8.9401  
1      0   0      0  1    8.9401   15876.0         8.9401  
2      0   0      0  1    8.9401   15876.0         8.9401  
3      0   0      0  1    8.9401   15876.0         8.9401  
4      0   0      0  1    8.9401   15876.0         8.9401

## Data wrangling

In [3]:
# Create rental_length_days column
rental_info['rental_date'] = pd.to_datetime(rental_info['rental_date'])
rental_info['return_date'] = pd.to_datetime(rental_info['return_date'])


rental_info["rental_length_days"] = (rental_info["return_date"]-rental_info["rental_date"]).dt.days
rental_info.head()

rental_date               return_date  amount  release_year  \
0 2005-05-25 02:54:33+00:00 2005-05-28 23:40:33+00:00    2.99        2005.0   
1 2005-06-15 23:19:16+00:00 2005-06-18 19:24:16+00:00    2.99        2005.0   
2 2005-07-10 04:27:45+00:00 2005-07-17 10:11:45+00:00    2.99        2005.0   
3 2005-07-31 12:06:41+00:00 2005-08-02 14:30:41+00:00    2.99        2005.0   
4 2005-08-19 12:30:04+00:00 2005-08-23 13:35:04+00:00    2.99        2005.0   

   rental_rate  length  replacement_cost                special_features  \
0         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
1         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
2         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
3         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   
4         2.99   126.0             16.99  {Trailers,"Behind the Scenes"}   

   NC-17  PG  PG-13  R  amount_2  length_2  rental_rate_2  rental_length_days  
0      0   0      0  1    8.9401   15876.0         8.9401                   3  
1      0   0      0  1    8.9401   15876.0         8.9401                   2  
2      0   0      0  1    8.9401   15876.0         8.9401                   7  
3      0   0      0  1    8.9401   15876.0         8.9401                   2  
4      0   0      0  1    8.9401   15876.0         8.9401                   4

In [4]:
rental_info["deleted_scenes"] = np.where(rental_info["special_features"].str.contains("Deleted Scenes"), 1, 0)
rental_info["behind_the_scenes"] = np.where(rental_info["special_features"].str.contains("Behind the Scenes"), 1, 0)
rental_info

rental_date               return_date  amount  \
0     2005-05-25 02:54:33+00:00 2005-05-28 23:40:33+00:00    2.99   
1     2005-06-15 23:19:16+00:00 2005-06-18 19:24:16+00:00    2.99   
2     2005-07-10 04:27:45+00:00 2005-07-17 10:11:45+00:00    2.99   
3     2005-07-31 12:06:41+00:00 2005-08-02 14:30:41+00:00    2.99   
4     2005-08-19 12:30:04+00:00 2005-08-23 13:35:04+00:00    2.99   
...                         ...                       ...     ...   
15856 2005-08-22 10:49:15+00:00 2005-08-29 09:52:15+00:00    6.99   
15857 2005-07-31 09:48:49+00:00 2005-08-04 10:53:49+00:00    4.99   
15858 2005-08-20 10:35:30+00:00 2005-08-29 13:03:30+00:00    8.99   
15859 2005-07-31 13:10:20+00:00 2005-08-08 14:07:20+00:00    7.99   
15860 2005-08-18 06:33:55+00:00 2005-08-24 07:14:55+00:00    5.99   

       release_year  rental_rate  length  replacement_cost  \
0            2005.0         2.99   126.0             16.99   
1            2005.0         2.99   126.0             16.99   
2            2005.0         2.99   126.0             16.99   
3            2005.0         2.99   126.0             16.99   
4            2005.0         2.99   126.0             16.99   
...             ...          ...     ...               ...   
15856        2009.0         4.99    88.0             11.99   
15857        2009.0         4.99    88.0             11.99   
15858        2009.0         4.99    88.0             11.99   
15859        2009.0         4.99    88.0             11.99   
15860        2009.0         4.99    88.0             11.99   

                                      special_features  NC-17  PG  PG-13  R  \
0                       {Trailers,"Behind the Scenes"}      0   0      0  1   
1                       {Trailers,"Behind the Scenes"}      0   0      0  1   
2                       {Trailers,"Behind the Scenes"}      0   0      0  1   
3                       {Trailers,"Behind the Scenes"}      0   0      0  1   
4                       {Trailers,"Behind the Scenes"}      0   0      0  1   
...                                                ...    ...  ..    ... ..   
15856  {Trailers,"Deleted Scenes","Behind the Scenes"}      0   0      0  1   
15857  {Trailers,"Deleted Scenes","Behind the Scenes"}      0   0      0  1   
15858  {Trailers,"Deleted Scenes","Behind the Scenes"}      0   0      0  1   
15859  {Trailers,"Deleted Scenes","Behind the Scenes"}      0   0      0  1   
15860  {Trailers,"Deleted Scenes","Behind the Scenes"}      0   0      0  1   

       amount_2  length_2  rental_rate_2  rental_length_days  deleted_scenes  \
0        8.9401   15876.0         8.9401                   3               0   
1        8.9401   15876.0         8.9401                   2               0   
2        8.9401   15876.0         8.9401                   7               0   
3        8.9401   15876.0         8.9401                   2               0   
4        8.9401   15876.0         8.9401                   4               0   
...         ...       ...            ...                 ...             ...   
15856   48.8601    7744.0        24.9001                   6               1   
15857   24.9001    7744.0        24.9001                   4               1   
15858   80.8201    7744.0        24.9001                   9               1   
15859   63.8401    7744.0        24.9001                   8               1   
15860   35.8801    7744.0        24.9001                   6               1   

       behind_the_scenes  
0                      1  
1                      1  
2                      1  
3                      1  
4                      1  
...                  ...  
15856                  1  
15857                  1  
15858                  1  
15859                  1  
15860                  1  

[15861 rows x 18 columns]

## Preprocessing 

In [5]:
X = rental_info.iloc[:,2:18]
X = X.drop(['rental_length_days','special_features'], axis=1)
y = rental_info['rental_length_days']

feature_names = X.columns
feature_names

Index(['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost',
       'NC-17', 'PG', 'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2',
       'deleted_scenes', 'behind_the_scenes'],
      dtype='object')

In [6]:
X.shape

(15861, 14)

## Using lasso to obtain key features

In [7]:
# Start preporcessing

# Slit training and test datasets

X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=9)

# Import Lasso
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.3)

# Fit the model to the data
lasso.fit(X_train, y_train)

# Compute and print the coefficients
lasso_coef = lasso.fit(X,y).coef_

print(lasso_coef)

# Get selected features
selected_features = X_train.columns[lasso_coef != 0]

print(selected_features)

[ 5.87567707e-01  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  4.32652862e-02  2.55377037e-06 -1.52154989e-01
 -0.00000000e+00  0.00000000e+00]
Index(['amount', 'amount_2', 'length_2', 'rental_rate_2'], dtype='object')


## Evaluating Models

In [8]:
##Evalaute the models

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Apply selected features to linear regression
linear_model = LinearRegression()
linear_model.fit(X_train[selected_features], y_train)
linear_predictions = linear_model.predict(X_test[selected_features])
linear_rmse = mean_squared_error(y_test, linear_predictions, squared=False)
print(f"Linear Regression RMSE: {linear_rmse:.4f}")

# Apply selected features to decision tree regression
tree_model = DecisionTreeRegressor()
tree_model.fit(X_train[selected_features], y_train)
tree_predictions = tree_model.predict(X_test[selected_features])
tree_rmse = mean_squared_error(y_test, tree_predictions, squared=False)
print(f"Decision Tree Regression RMSE: {tree_rmse:.4f}")


# Apply selected features to random forest regression
forest_model = RandomForestRegressor()
forest_model.fit(X_train[selected_features], y_train)
forest_predictions = forest_model.predict(X_test[selected_features])
forest_rmse = mean_squared_error(y_test, forest_predictions, squared=False)
print(f"Random Forest Regression RMSE: {forest_rmse:.4f}")

Linear Regression RMSE: 1.7585
Decision Tree Regression RMSE: 1.5550
Random Forest Regression RMSE: 1.5467


## Obtaining Best Model

In [9]:
# Save the model with the lowest RMSE as best_mse
best_model = None
best_rmse = float('inf')

if linear_rmse < best_rmse:
    best_model = linear_model
    best_rmse = linear_rmse

if tree_rmse < best_rmse:
    best_model = tree_model
    best_rmse = tree_rmse

if forest_rmse < best_rmse:
    best_model = forest_model
    best_rmse = forest_rmse


print(f"Best Model RMSE: {best_rmse:.4f}")



Best Model RMSE: 1.5467


## Answer

In [10]:
best_mse = 1.5467